## 1) Imports & paths
##### 💡 Concept
Before manipulating data, we must set up the environment — define where data lives and where cleaned outputs will go. Think of this like laying out your lab tools before starting an experiment. Without consistent paths, reproducibility collapses.

In [1]:
import pandas as pd, numpy as np, re, json
from pathlib import Path
RNG = 42

ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
DATA_RAW = ROOT / "data" / "raw"
DATA_PROC = ROOT / "data" / "processed"
REPORTS = ROOT / "reports"
FIGS = ROOT / "outputs" / "figures"

DATA_PROC.mkdir(parents=True, exist_ok=True)
REPORTS.mkdir(parents=True, exist_ok=True)
FIGS.mkdir(parents=True, exist_ok=True)

## 2) Load raw datasets

##### 💡 Concept
At its core, “loading” isn’t just reading CSVs — it’s about validating the data contract.
Each dataset has a schema (columns, types, units). Before merging, you must check whether those schemas align or conflict.

In [ ]:
df1 = pd.read_csv(DATA_RAW / "dataset1_data_science_job.csv")
df2 = pd.read_csv(DATA_RAW / "dataset2_all_job_post.csv")
df3 = pd.read_csv(DATA_RAW / "dataset3_ai_job_dataset.csv")

for name, df in {"df1": df1, "df2": df2, "df3": df3}.items():
    print(name, df.shape); display(df.head(2)); display(df.info())

## 3) Profiling snapshot (lightweight)

##### 💡 Concept
Profiling is the diagnostic stage of cleaning — like running blood tests before prescribing medicine.
It tells you what’s wrong: missing values, strange datatypes, duplicates, etc.
Without this, cleaning becomes random guessing.

In [ ]:
def profile(df: pd.DataFrame, name: str) -> dict:
    """Return basic profile stats for df. 
    Time: O(n * c). Space: O(c)."""
    return {
        "rows": len(df),
        "cols": df.shape[1],
        "na_counts": df.isna().sum().to_dict(),
        "dup_rows": int(df.duplicated().sum()),
        "numeric_cols": df.select_dtypes(include="number").columns.tolist(),
        "object_cols": df.select_dtypes(include="object").columns.tolist(),
    }

profiles = {k: profile(v, k) for k, v in {"df1": df1, "df2": df2, "df3": df3}.items()}
print(json.dumps(profiles, indent=2)[:2000], "...")


## 4️) Schema Harmonization

##### 💡 Concept
Datasets from different sources often call the same thing by different names — e.g., job_title vs title.
Before merging, we need a shared vocabulary.
This is the lingua franca of your data — making sure everyone (and every dataset) “speaks the same language”.

In [6]:
COLMAP = {
    "title": "job_title",
    "jobTitle": "job_title",
    "category": "job_category",          # NEW
    "skills": "required_skills",
    "job_skill_set": "required_skills",  # NEW
    "experience": "experience_level",
    "exp_level": "experience_level",
    "salary_in_usd": "salary_usd",       # NEW
    "salary_usd": "salary_usd",
    "salaryLocal": "salary_local",       # just in case
    "salary": "salary",
    "location": "company_location",
    "posted_at": "posting_date",
}

DATE_COLS = ["posting_date", "application_deadline"]
NUM_COLS  = ["salary_usd", "salary", "salary_local",
             "remote_ratio", "years_experience",
             "benefits_score", "job_description_length"]

def harmonize(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize schema + dtypes. Time: O(n+c), Space: ~O(1) extra."""
    out = df.rename(columns={k:v for k,v in COLMAP.items() if k in df.columns}).copy()
    for d in DATE_COLS:
        if d in out.columns:
            out[d] = pd.to_datetime(out[d], errors="coerce")
    for n in NUM_COLS:
        if n in out.columns:
            out[n] = pd.to_numeric(out[n], errors="coerce")
    return out

df1h, df2h, df3h = map(harmonize, (df1, df2, df3))


## 5️) Missing Values & “Unknown” Categories

##### 💡 Concept
Missing data is information — it tells you where the system failed to observe.  
We never randomly “fill” it; we reason about why it’s missing.  
Duplicates distort truth — one job posted twice looks like double demand.  
Here, imputation = an informed guess.


In [7]:
def fill_missing(df):
    out = df.copy()
    for c in out.select_dtypes("object"):
        out[c] = out[c].fillna("Unknown")
    for c in out.select_dtypes("number"):
        out[c] = out[c].fillna(out[c].median())
    return out.drop_duplicates()

df1c, df2c, df3c = map(fill_missing, (df1h, df2h, df3h))


## 6) De-dupe & Validity Checks (ID rules, logical ranges)

##### 💡 Concept
Cleaning is not just about fixing missing or noisy data — it’s about protecting logical truth in your dataset.
Before modeling or visualization, you must ensure that:

No duplicate rows distort your results.

Key fields (like job title, salary, or experience) obey reasonable rules.

No impossible records exist (like “-5 years experience” or “posted in 2030”).

In [8]:
def clean_dupes_ranges(df: pd.DataFrame) -> pd.DataFrame:
    """
    De-dupe using strongest key available, then clamp obviously invalid ranges
    and lightly winsorize salary-like columns.
    Time: ~O(n). Space: ~O(n) due to copy/masks.
    """
    import numpy as np
    out = df.copy()

    # --- 1) De-dupe ---
    if "job_id" in out.columns:
        out = out.drop_duplicates(subset=["job_id"])
    else:
        cand_keys = ["job_title", "company_name", "company_location", "posting_date"]
        subset = [c for c in cand_keys if c in out.columns]
        # If nothing is available, fall back to full-row dedupe
        out = out.drop_duplicates(subset=subset if subset else None)

    # Optional: normalize posting_date to date for dedupe consistency upstream
    # if "posting_date" in out.columns and np.issubdtype(out["posting_date"].dtype, np.datetime64):
    #     out["posting_date"] = out["posting_date"].dt.normalize()

    # --- 2) Hard bounds (domain rules) ---
    bounds = {
        "remote_ratio": (0, 100),
        "years_experience": (0, 50),
        "job_description_length": (0, None),  # min 0
        "benefits_score": (0, 100),           # adjust later if your canonical scale is 0..1
    }
    for col, (lo, hi) in bounds.items():
        if col in out.columns:
            # coerce numerics defensively
            out[col] = pd.to_numeric(out[col], errors="coerce")
            if lo is not None:
                out[col] = out[col].clip(lower=lo)
            if hi is not None:
                out[col] = out[col].clip(upper=hi)

    # --- 3) Light winsorization for salaries ---
    for s in ("salary_usd", "salary", "salary_local"):
        if s in out.columns:
            out[s] = pd.to_numeric(out[s], errors="coerce")
            ser = out[s].dropna()
            if ser.size >= 10:  # need enough data to estimate tails
                q = ser.quantile([0.005, 0.995])
                low, high = float(q.iloc[0]), float(q.iloc[1])
                # If quantiles are nan (pathological), skip safely
                if np.isfinite(low) and np.isfinite(high) and low <= high:
                    out[s] = out[s].clip(lower=low, upper=high)
            # Enforce strictly positive salaries (scraper glitches)
            out[s] = out[s].clip(lower=1)

    return out


## 7) Outlier handling (salary via IQR within job_title×location)

##### 💡 Concept

Outliers aren’t “wrong,” they’re rare. We cap them only when they distort aggregates/models. Do it within comparable groups (e.g., job_title × company_location) so a Bay Area Staff ML salary doesn’t clamp a Pune junior role.

In [9]:
def cap_salary_iqr(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cap salary_usd outliers per (job_title × company_location) using IQR.
    Guards: dtype coercion, min group size, two-tailed fences, NaN safety.
    Time: ~O(n). Space: ~O(n).
    """
    import numpy as np
    out = df.copy()

    if "salary_usd" not in out.columns:
        return out

    # ensure numeric
    out["salary_usd"] = pd.to_numeric(out["salary_usd"], errors="coerce")

    grp_keys = [k for k in ["job_title", "company_location"] if k in out.columns]
    if not grp_keys:
        # No grouping keys → optional global cap (or just return)
        return out

    # optional: drop rows with all-NaN salary before stats
    valid = out.dropna(subset=["salary_usd"])

    # min group size mask
    sizes = valid.groupby(grp_keys)["salary_usd"].size()
    big_groups = sizes[sizes >= 8]  # tweak threshold as needed

    if big_groups.empty:
        return out

    big_idx = big_groups.index  # MultiIndex of groups

    # compute fences on big groups only
    q1 = valid[valid.set_index(grp_keys).index.isin(big_idx)] \
             .groupby(grp_keys)["salary_usd"].quantile(0.25)
    q3 = valid[valid.set_index(grp_keys).index.isin(big_idx)] \
             .groupby(grp_keys)["salary_usd"].quantile(0.75)

    iqr = (q3 - q1)
    lo = (q1 - 1.5 * iqr).rename("lo")
    hi = (q3 + 1.5 * iqr).rename("hi")

    # join fences back by group keys (handles MultiIndex)
    fences = pd.concat([lo, hi], axis=1)
    out = out.join(fences, on=grp_keys)

    # cap where fences exist; keep original where fences are NaN (small groups, NaNs)
    has_lo = out["lo"].notna()
    has_hi = out["hi"].notna()
    out.loc[has_lo, "salary_usd"] = np.maximum(out.loc[has_lo, "salary_usd"], out.loc[has_lo, "lo"])
    out.loc[has_hi, "salary_usd"] = np.minimum(out.loc[has_hi, "salary_usd"], out.loc[has_hi, "hi"])

    # final sanity: enforce strictly positive salaries
    out["salary_usd"] = out["salary_usd"].clip(lower=1)

    return out.drop(columns=[c for c in ["lo", "hi"] if c in out.columns])


## 8) Skill standardization (lowercase, split, synonyms)

##### 💡 Concept

Skills are messy — “Python”, “python3”, “PYTHON” — but all mean the same ability.
We normalize by lowercasing, removing punctuation, and mapping synonyms.
This step ensures that your model doesn’t think “JS” and “JavaScript” are different universes.

In [ ]:
# ============================================
# Skill Normalization — CSV-aware & Robust
# Time:  O(n * k)  |  Space: O(n * k)
# n = rows, k = avg skills per row
# ============================================
import re
import pandas as pd

# 0) Helper: pick the right raw skill column (dataset2 uses 'job_skill_set')
def _get_skills_series(df: pd.DataFrame) -> pd.Series:
    if "required_skills" in df.columns:
        return df["required_skills"]
    if "job_skill_set" in df.columns:
        # normalize name so downstream is consistent
        df = df.copy()
        df["required_skills"] = df["job_skill_set"]
        return df["required_skills"]
    # fallback: empty strings
    return pd.Series([""] * len(df), index=df.index)

# 1) Synonym map — extend as you discover variants in your EDA
SYN = {
    # Python family
    "python3": "python", "py": "python",
    # JavaScript family
    "js": "javascript", "ecmascript": "javascript",
    "node.js": "nodejs", "node js": "nodejs", "node": "nodejs",
    "ts": "typescript",
    # C-family
    "c#": "csharp", "c-sharp": "csharp", "c sharp": "csharp",
    "cpp": "c++", "c plus plus": "c++",
    # Data/ML libs
    "sklearn": "scikit-learn", "scikit learn": "scikit-learn",
    "tf": "tensorflow", "tf2": "tensorflow",
    # DBs
    "postgresql": "postgres", "psql": "postgres",
    # FE frameworks
    "react.js": "react", "reactjs": "react",
    "vue.js": "vue", "vuejs": "vue",
    # Clouds
    "amazon web services": "aws", "aws cloud": "aws",
    "ms azure": "azure", "gcp": "google cloud",
}

# 2) Tiny normalizer helpers
_version_pat = re.compile(r"\s*\d+([._-]\d+)*$")     # strips trailing versions: "python 3.11" -> "python"
_delim_pat   = re.compile(r"[|,/\s]+")

def _canon(token: str) -> str:
    t = token.strip().lower()
    if not t:
        return ""
    # unify some punctuation variants
    t = t.replace("–", "-").replace("—", "-").replace("_", " ")
    # drop trailing version numbers
    t = _version_pat.sub("", t).strip()
    # apply dictionary
    return SYN.get(t, t)

def normalize_skills_cell(cell) -> list[str]:
    """Return sorted unique canonical skills from one cell."""
    if pd.isna(cell) or not str(cell).strip():
        return []
    tokens = _delim_pat.split(str(cell).lower())
    cleaned = [_canon(tok) for tok in tokens if tok and tok.strip()]
    # drop empties, uniquify, sort for reproducibility
    return sorted({t for t in cleaned if t})

# 3) Apply to each of your dataframes (df1c, df2c, df3c)
for df in (df1c, df2c, df3c):
    skills_series = _get_skills_series(df)
    df["required_skills"] = skills_series  # ensures the column exists uniformly
    df["required_skills_norm"] = skills_series.apply(normalize_skills_cell)
    # Optional: BI-friendly string column (pipe-joined)
    df["required_skills_norm_str"] = df["required_skills_norm"].apply(lambda xs: "|".join(xs))

# 4) (Optional) Quick sanity peek — comment out in final notebook
def _preview(df, name):
    if {"required_skills","required_skills_norm"}.issubset(df.columns):
        sample = df.loc[df["required_skills"].notna(), ["required_skills","required_skills_norm"]].head(5)
        print(f"\n[{name}] skill normalization preview:")
        for _, row in sample.iterrows():
            print("  raw ->", row["required_skills"])
            print("  norm->", row["required_skills_norm"])

_preview(df1c, "df1c")
_preview(df2c, "df2c")
_preview(df3c, "df3c")


## 9) Minimal Feature Engineering (salary_midpoint, posting_year, skill_count)

##### 💡 Concept

Feature engineering converts raw facts into signals.
By extracting date parts or skill counts, we help models see structure in what was previously unstructured.
For instance, knowing “posting_month” helps detect hiring seasonality.

In [18]:
import pandas as pd
import numpy as np

def _len_safe(x):
    return len(x) if isinstance(x, (list, tuple)) else 0

def add_basic_features(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    # 1) Salary midpoint (coerce if needed)
    if {"salary_min", "salary_max"}.issubset(out.columns):
        out["salary_min"] = pd.to_numeric(out["salary_min"], errors="coerce")
        out["salary_max"] = pd.to_numeric(out["salary_max"], errors="coerce")
        out["salary_midpoint"] = (out["salary_min"] + out["salary_max"]) / 2
    elif "salary_usd" in out.columns:
        out["salary_midpoint"] = pd.to_numeric(out["salary_usd"], errors="coerce")
    elif "salary" in out.columns:
        out["salary_midpoint"] = pd.to_numeric(out["salary"], errors="coerce")

    # 2) Posting date breakdown (ensure datetime first)
    if "posting_date" in out.columns:
        if not np.issubdtype(out["posting_date"].dtype, np.datetime64):
            out["posting_date"] = pd.to_datetime(out["posting_date"], errors="coerce")
        out["posting_year"] = out["posting_date"].dt.year
        out["posting_month"] = out["posting_date"].dt.month
        # optional:
        # out["posting_quarter"] = out["posting_date"].dt.quarter
        # out["is_recent"] = out["posting_date"] >= (pd.Timestamp.now() - pd.Timedelta(days=90))

    # 3) Skill count (robust to non-lists)
    if "required_skills_norm" in out.columns:
        out["skill_count"] = out["required_skills_norm"].apply(_len_safe)
    else:
        out["skill_count"] = 0

    return out

# Apply to all three cleaned dataframes
df1f, df2f, df3f = map(add_basic_features, (df1c, df2c, df3c))


In [19]:
print(df1f[["salary_midpoint","posting_year","posting_month","skill_count"]].head())


KeyError: "['posting_year', 'posting_month'] not in index"

## 10) Data-Quality Metrics (completeness, duplicates, validity)

##### 💡 Concept

Once the dataset looks “clean,” we must prove it.
Data quality is the empirical backbone of credibility. We quantify how trustworthy our cleaned data is instead of assuming it’s fine.

In [16]:
# ===============================
# 🔎 STEP 10 — Data Quality Metrics (Robust)
# Time: O(n·c) | Space: O(c)
# ===============================

import json
import numpy as np
import pandas as pd
from pathlib import Path

# Make sure these exist
# master   → your merged DataFrame (from Step 9)
# REPORTS  → Path to your reports folder (created in Step 1)

def write_data_quality_report(master: pd.DataFrame, reports_dir: Path) -> dict:
    """Generate and save a data-quality summary for the cleaned dataset."""
    reports_dir = Path(reports_dir)
    dq = {
        "rows": int(len(master)),
        "cols": int(master.shape[1]),
    }

    # --- Duplicates ---
    dq["duplicates"] = int(master.duplicated().sum())

    # --- Completeness (% non-null per column) ---
    completeness = {}
    list_nonempty = {}
    for c in master.columns:
        comp = float(master[c].notna().mean()) if len(master) else 0.0
        completeness[c] = round(comp, 4)
        # list-like columns (e.g., required_skills_norm)
        if master[c].dtype == "object" and master[c].apply(lambda x: isinstance(x, (list, tuple))).any():
            nonempty = float(master[c].apply(lambda x: isinstance(x, (list, tuple)) and len(x) > 0).mean())
            list_nonempty[c] = round(nonempty, 4)

    dq["completeness"] = completeness
    if list_nonempty:
        dq["list_nonempty_ratio"] = list_nonempty

    # --- Validity metrics ---
    validity = {}

    # Date sanity
    if "posting_date" in master.columns:
        pd_coerced = pd.to_datetime(master["posting_date"], errors="coerce")
        in_range = (pd_coerced >= pd.Timestamp("2010-01-01")) & (pd_coerced <= pd.Timestamp("2025-12-31"))
        validity["posting_date_in_range"] = float(in_range.mean())

    # Salary positivity
    if "salary_usd" in master.columns:
        sal = pd.to_numeric(master["salary_usd"], errors="coerce")
        validity["salary_usd_positive"] = float((sal > 0).mean())

    # Remote ratio within 0–100
    if "remote_ratio" in master.columns:
        rr = pd.to_numeric(master["remote_ratio"], errors="coerce")
        validity["remote_ratio_0_100"] = float(((rr >= 0) & (rr <= 100)).mean())

    # Years experience 0–50
    if "years_experience" in master.columns:
        ye = pd.to_numeric(master["years_experience"], errors="coerce")
        validity["years_experience_0_50"] = float(((ye >= 0) & (ye <= 50)).mean())

    dq["validity"] = validity

    # --- Numeric quick stats (min, max, missing%) ---
    numeric_ranges = {}
    for c in master.select_dtypes(include=[np.number]).columns:
        s = master[c]
        if s.notna().any():
            numeric_ranges[c] = {
                "min": float(np.nanmin(s)),
                "max": float(np.nanmax(s)),
                "pct_missing": round(float(s.isna().mean()), 4)
            }
        else:
            numeric_ranges[c] = {"min": None, "max": None, "pct_missing": 1.0}
    dq["numeric_ranges"] = numeric_ranges

    # --- Save JSON ---
    reports_dir.mkdir(parents=True, exist_ok=True)
    out_path = reports_dir / "data_quality_report.json"
    with open(out_path, "w") as f:
        json.dump(dq, f, indent=2)

    print(f"✅ Data-Quality Report written → {out_path}")
    return dq

# ✅ CALL IT HERE
dq = write_data_quality_report(master, REPORTS)


NameError: name 'master' is not defined

## 11) Save Artifacts & Summary (What Changed · Counts Before/After)

##### 💡 Concept

Science isn’t just results—it’s reproducible results.
Saving artifacts ensures any other analyst can rebuild your exact cleaned dataset tomorrow with zero guesswork.
